# Identify Worker Labeling Efficiency using SageMaker GroundTruth

### Introduction
Welcome to our example on identifying worker labeling efficiency for a SageMaker GroundTruth Labeling job. Before running this notebook, please make sure that all the instructions prior to the section 'Setup the Automated Accuracy Logic' from this [blog](https://aws.amazon.com/blogs/machine-learning/identifying-worker-labeling-efficiency-using-amazon-sagemaker-ground-truth/) have been followed. 

We first walk through some permissions and utility methods required for setting up. We then follow up with the logic of extracting individual worker responses and annotations from the output manifest and golden manifest to automatically calculate worker accuracy.

### Import the necessary libraries

In [ ]:
import json
import pprint
import boto3
from datetime import datetime

%matplotlib inline
from urllib.parse import urlparse
import pandas as pd
import numpy as np
import sagemaker
from sagemaker import get_execution_role

### Permissions

As a sanity check, let us verify that this notebook is in the same region as the s3 bucket setup earlier following the instructions in the blog. 

In [ ]:
def run_sanity_check(bucket_name):
    role = get_execution_role()
    region = boto3.session.Session().region_name
    s3 = boto3.client("s3")
    bucket_region = s3.get_bucket_location(Bucket=bucket_name)["LocationConstraint"]
    if bucket_region is None:
        bucket_region = "us-east-1"
    assert (
        bucket_region == region
    ), "Your S3 bucket {} and this notebook need to be in the same region.".format(bucket_name)

### Initialize the s3 client

In [ ]:
s3_client = boto3.client("s3")

### Utility methods

In [ ]:
def get_bucket_keys(bucket_name, prefix):
    keys = []
    paginator = s3_client.get_paginator("list_objects")
    operation_parameters = {"Bucket": bucket_name, "Prefix": prefix}
    page_iterator = paginator.paginate(**operation_parameters)
    for page in page_iterator:
        for obj in page["Contents"]:
            keys.append(obj["Key"])
    return keys

In [ ]:
def get_bucket_key_from_s3_uri(s3_path):
    parsed_url = urlparse(s3_path, allow_fragments=False)
    bucket = parsed_url.netloc
    key = parsed_url.path.lstrip("/")
    return bucket, key

### Calculate worker accuracy
Given the worker annotation and the true annotation, we calculate the accuracy of the worker. -1 is returned if we
do not know the true annotation.

In [ ]:
def get_worker_accuracy(worker_annotation, true_annotation):
    if true_annotation is None:
        return -1
    if worker_annotation == true_annotation:
        return 1
    else:
        return 0

### Extract and process responses from golden manifest file and output manifest file.
For each data object, we fetch the worker response, the golden manifest annotation (if available) and the output manifest annotation. We then calculate the worker accuracy against the output manifest annotation and the golden manifest annotation. This is stored in a dictionary indexed by worker ID.

In [ ]:
def extract_ground_truth_file(ground_truth_file):
    dict_manifest_index_gt = {}
    if ground_truth_file is not None:
        bucket_name, key = get_bucket_key_from_s3_uri(ground_truth_file)
        content = s3_client.get_object(Bucket=bucket_name, Key=key)["Body"].read().decode("utf-8")
        content_list = content.split("\n")
        for x in content_list:
            if x:
                dict_ = json.loads(x)
                for key_, value_ in dict_.items():
                    dict_manifest_index_gt[key_] = value_
    return dict_manifest_index_gt

In [ ]:
def extract_consolidated_file(consolidated_file, labeling_job_name):
    dict_manifest_index_consolidated = {}
    if consolidated_file is not None:
        i = 0
        bucket_name, key = get_bucket_key_from_s3_uri(consolidated_file)
        content = s3_client.get_object(Bucket=bucket_name, Key=key)["Body"].read().decode("utf-8")
        content_list = content.split("\n")
        if len(content_list) > 0:
            metadata_name = list(json.loads(content_list[0]).keys())[-1]
        for x in content_list:
            if x:
                dict_manifest_index_consolidated[str(i)] = json.loads(x)[metadata_name][
                    "class-name"
                ]
                i += 1
    return dict_manifest_index_consolidated

In [ ]:
def process_raw_annotation(worker_response):
    raw_annotation = worker_response["answerContent"]["crowd-classifier"]["label"]
    return raw_annotation

In [ ]:
def process_worker_responses(
    bucket_name, all_keys_worker_response, dict_manifest_index_gt, dict_manifest_index_consolidated
):
    dict_worker_response = {}
    for worker_response_key in all_keys_worker_response:
        if worker_response_key.endswith(".json"):
            response_content = json.loads(
                s3_client.get_object(Bucket=bucket_name, Key=worker_response_key)["Body"]
                .read()
                .decode("utf-8")
            )
            manifest_index = worker_response_key.split("/")[-2]
            list_worker_response = response_content["answers"]
            for worker_response in list_worker_response:
                processed_annotation = process_raw_annotation(worker_response)
                ground_truth_annotation = None
                consolidated_annotation = None
                if manifest_index in dict_manifest_index_gt:
                    ground_truth_annotation = dict_manifest_index_gt[manifest_index]
                consolidated_annotation = dict_manifest_index_consolidated[manifest_index]
                worker_accuracy_gt = get_worker_accuracy(
                    processed_annotation, ground_truth_annotation
                )
                worker_accuracy_consolidated = get_worker_accuracy(
                    processed_annotation, consolidated_annotation
                )
                worker_id = worker_response["workerId"]
                if worker_id not in dict_worker_response:
                    dict_worker_response[worker_id] = []
                dict_worker_response[worker_id].append(
                    {
                        "ManifestIndex": manifest_index,
                        "GTAccuracy": worker_accuracy_gt,
                        "ConsolidatedAccuracy": worker_accuracy_consolidated,
                    }
                )

    return dict_worker_response

### Calculate the worker accuracy metrics and write to the given output location 
This method is the starting point to this script and invokes other methods defined above. A sample invocation is shown in the following cells.

In [ ]:
def write_worker_metrics(
    labeling_job_output_location,
    labeling_job_name,
    s3_output_worker_metrics,
    ground_truth_file=None,
):
    # read the ground truth file and form a dict
    dict_manifest_index_gt = extract_ground_truth_file(ground_truth_file)
    if not labeling_job_output_location.endswith("/"):
        labeling_job_output_location = labeling_job_output_location + "/"
    output_manifest = (
        labeling_job_output_location + labeling_job_name + "/manifests/output/output.manifest"
    )

    # read the output manifest and form a dict
    dict_manifest_index_consolidated = extract_consolidated_file(output_manifest, labeling_job_name)

    # get the raw worker annotations for each data object id
    worker_response_root = (
        labeling_job_output_location + labeling_job_name + "/annotations/worker-response"
    )
    bucket_name, key = get_bucket_key_from_s3_uri(worker_response_root)

    all_keys_worker_response = get_bucket_keys(bucket_name, key)

    worker_response_dict = process_worker_responses(
        bucket_name,
        all_keys_worker_response,
        dict_manifest_index_gt,
        dict_manifest_index_consolidated,
    )
    output_dict = {}

    for worker_id, accuracy_list in worker_response_dict.items():
        list_gt_accuracy = []
        list_consolidated_accuracy = []
        for y_ in accuracy_list:
            list_gt_accuracy.append(y_["GTAccuracy"])
            list_consolidated_accuracy.append(y_["ConsolidatedAccuracy"])
        num_data_objects_golden = len(list_gt_accuracy) - list_gt_accuracy.count(-1)
        list_gt_accuracy_only_annotated = [x for x in list_gt_accuracy if x != -1]

        if len(list_gt_accuracy_only_annotated) == 0:
            avg_gt_accuracy = 0
        else:
            avg_gt_accuracy = np.mean(list_gt_accuracy_only_annotated)

        avg_consolidated_accuracy = np.mean(list_consolidated_accuracy)
        num_data_objects = len(accuracy_list)
        output_dict[worker_id] = {
            "Total Number Of Objects Annotated": num_data_objects,
            "Number Of Golden Standard Objects Annotated": num_data_objects_golden,
            "Average Accuracy Compared To Other Workers": avg_consolidated_accuracy,
            "Average Golden Standard Accuracy": avg_gt_accuracy,
        }

    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(output_dict)
    bucket_name, key = get_bucket_key_from_s3_uri(s3_output_worker_metrics)
    s3_client.put_object(Body=json.dumps(output_dict), Bucket=bucket_name, Key=key)

### Below is a sample invocation of the script.  Please replace with your own inputs.
* **bucket**: Please provide your bucket name. Our example bucket name is 'gec-sagemaker-blog'.<br>


* **labeling_job_output_location**:  Please make sure this is the same as provided while creating the labeling job. In our example this is 's3://gec-sagemaker-blog/output'.<br>
* **labeling_job_name**: We can get the labeling job name from the Labeling jobs section under the SageMaker console. <br>


* **golden_answers**: This is the golden manifest file. Each line is a key value pair in JSON format. The key is the line number of the data object in the input manifest file provided while creating the labeling job. The value is the true label of the data object. This is an optional parameter. <br>


* **worker_metrics_output**: This s3 path of the output JSON file storing worker metrics. This file gets generated automatically. In our example this is 's3://gec-sagemaker-blog/worker_metrics.json'. Each entry in this JSON file is of the form where `public.us-east-1.PDC` is the worker ID, the **total number of objects** annotated by this worker is 3. We know the **true labels** for 2 out of these 3 objects. The golden manifest file **need not** store the true labels of all data objects. The worker correctly annotated the 2 data objects present in the golden manifest file with respect to the true annotation. The worker agreed with other workers for all 3 data objects.

        
        'public.us-east-1.PDC': {
                'Total Number Of Objects Annotated': 3,
                'Number Of Golden Standard Objects Annotated': 2,
                'Average Golden Standard Accuracy': 1.0,
                'Average Accuracy Compared To Other Workers': 1.0,
        }
       

In [ ]:
# Sample to invoke the script. Please replace with own inputs
bucket = "gec-sagemaker-blog"
bucket_root_url = "s3://" + bucket
golden_answers = bucket_root_url + "/" + "golden.manifest"
worker_metrics_output = bucket_root_url + "/" + "worker_metrics.json"
labeling_job_name = "person-animal-plant"
labeling_job_output_location = bucket_root_url

We first run a sanity check to ensure that the bucket setup earlier as part of the blog instructions exists in the
 same region as this notebook. If the assertion fails, please cross check that the notebook is in the same region as the bucket. We then call the write_worker_metrics function with the inputs defined above.

In [ ]:
run_sanity_check(bucket)
write_worker_metrics(
    labeling_job_output_location, labeling_job_name, worker_metrics_output, golden_answers
)

### Conclusion

In this notebook, we walked through analyzing the accuracy of workers for a completed Text Classification labeling job. We calculated the worker accuracy against the consoldiated output annotation and the golden annotation (when available). This shows us a measure of how often a worker agrees with the other workers as well as the the correctness of a worker with respect to the true annotation.

This script is setup to calculate worker accuracy for our classification modalities. However, it is possible to use a similar logic for other Ground Truth modalities as well. We use different accuracy metrics for different modalities, so code snippet calculating the worker accuracy metric would likely need to be updated. In case of classification, it is 0 or 1 but for modalities like bounding box we could use metrics like Intersection over Union. Please note that other utility methods like processing the worker responses will also need to be updated accordingly. 
